# Summary

There are acceptable and un-acceptable ways of flattening lists of dicts, if you want to use the result in autograd.

See below. 

https://github.com/HIPS/autograd/blob/master/docs/tutorial.md#tldr-do-use

In [1]:
import numpy as np
import autograd.numpy as ag_np
import autograd

# Unsafe version for autograd. Uses indexing 

In [2]:
def flatten(bnn_param_list):
    n_dims = ag_np.sum([[arr.size for arr in layer.values()] for layer in bnn_param_list])
    flat_vec = ag_np.zeros(n_dims)
    start = 0
    for layer in bnn_param_list:
        stop = start + layer['w'].size
        flat_vec[start:stop] = layer['w'].flatten()  # <- BAD
        start = stop
        
        stop = start + layer['b'].size
        flat_vec[start:stop] = layer['b'].flatten()  # <- BAD
        start = stop
    return flat_vec

In [3]:
def sum_of_params(bnn_param_list):
    return ag_np.sum(flatten(bnn_param_list))

In [4]:
bnn_param_list = [dict(w=np.random.randn(3), b=np.random.randn(3))]

In [5]:
bnn_param_list

[{'w': array([-0.44944486,  2.10795451, -0.47880287]),
  'b': array([-0.05945471,  1.08620764, -1.2888056 ])}]

In [6]:
sum_of_params(bnn_param_list)

0.9176541056490732

In [7]:
g = autograd.grad(sum_of_params)

## Try to get gradient (will give error)

In [8]:
g(bnn_param_list)

ValueError: setting an array element with a sequence.

# SAFE version for autograd. Uses hstack rather than slice indexing

In [9]:
def flatten__safe(bnn_param_list):
    flat_arr_list = []
    for layer in bnn_param_list:
        flat_arr_list.append(layer['w'].flatten())
        flat_arr_list.append(layer['b'].flatten())
    return ag_np.hstack(flat_arr_list)

In [10]:
def sum_of_params__safe(bnn_param_list):
    return ag_np.sum(flatten__safe(bnn_param_list))

In [11]:
g = autograd.grad(sum_of_params__safe)

## Try to get gradient (will work!)

In [12]:
g(bnn_param_list)

[{'w': array([1., 1., 1.]), 'b': array([1., 1., 1.])}]